<a href="https://colab.research.google.com/github/rishikesh1410/Projects/blob/master/RecommenderSystemPyTorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
from __future__ import print_function, division

# Torch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Libraries
import numpy as np
import time
import os
import copy
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from numpy import *
from scipy import optimize
np.random.seed(1)

In [0]:
os.chdir("/gdrive/My Drive/data/movielens")

In [0]:
movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv')
rating_with_name = pd.read_csv('rating_with_name.csv')

In [0]:
user_prefs = pd.read_csv('new_user_prefs.csv')
movie_features = pd.read_csv('new_movie_features.csv')

In [0]:
movie_features = movie_features.iloc[:,1:]
user_prefs = user_prefs.iloc[:,1:]

In [7]:
movie_features

,0,1,2
0,0.969737,-0.504606,-0.759909
1,0.157291,-0.405990,-0.527766
2,0.440388,-1.066609,0.059473
3,0.820170,-1.179844,0.111779
4,1.018596,-1.267757,0.503781
5,0.540590,-0.343280,0.246707
6,-0.008349,-0.525246,-0.230984
7,0.319073,-1.218923,-0.746284
8,0.652205,-0.697770,0.198391
9,0.431078,-1.042976,-0.454492


In [8]:
user_prefs

,0,1,2
0,0.100705,-0.912991,-0.359637
1,-0.465897,-0.258130,-0.088400
2,0.337208,0.086550,0.549299
3,-0.655845,-1.108610,-0.649265
4,-0.372976,-0.600798,-0.030201
5,0.033659,-0.745804,-0.497728
6,-0.156978,-0.350801,-0.328748
7,-0.316445,0.118591,-0.440361
8,0.306367,-0.076792,0.137638
9,0.510554,0.459274,0.385452


In [0]:
movie_features = np.array(movie_features)
user_prefs = np.array(user_prefs)

In [0]:
def loss(movie_features, user_prefs, ratings, did_rate):

  cost = torch.sum( ((torch.mm(movie_features, torch.t(user_prefs)) * did_rate) - ratings) ** 2 ) / (1700*1000)
  return cost

In [0]:
def tabular_preview(ratings, n=15):
  
    user_groups = ratings.groupby('userId')['rating'].count()
    top_users = user_groups.sort_values(ascending=False)[:1000]

    movie_groups = ratings.groupby('movieId')['rating'].count()
    top_movies = movie_groups.sort_values(ascending=False)[:1700]

    top = (
        ratings.
        join(top_movies, rsuffix='_r', how='inner', on='movieId').
        join(top_users, rsuffix='_r', how='inner', on='userId'))

    return pd.crosstab(top.movieId, top.userId, top.rating, aggfunc=np.sum)

In [0]:
frame = tabular_preview(rating)

In [0]:
train_data = np.array(frame)

In [0]:
did_rate = np.ndarray(shape = (1700,1000))

In [0]:
for i in range(1700):
  for j in range(1000):
    if np.isnan(train_data[i][j]):
      train_data[i][j]=0

In [0]:
did_rate = (train_data != 0 )* 1

In [0]:
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]

    ratings_mean = np.zeros(shape = (num_movies, 1))
    ratings_norm = np.zeros(shape = ratings.shape)

    for i in range(num_movies): 
        idx = np.where(did_rate[i] == True)[0]
        ratings_mean[i] = np.mean(ratings[i, idx])
        ratings_norm[i, idx] = ratings[i, idx] - ratings_mean[i]

    return ratings_norm, ratings_mean

In [0]:
rating, rating_mean = normalize_ratings(train_data, did_rate)

In [0]:
movie_features = torch.tensor(movie_features, requires_grad=True)
user_prefs = torch.tensor(user_prefs, requires_grad=True)
did_rate = torch.tensor(did_rate, dtype=torch.double, requires_grad=False)
rating = torch.tensor(rating, requires_grad=False)
rating_mean = torch.tensor(rating_mean, dtype=torch.double, requires_grad=False)

In [20]:
loss(movie_features, user_prefs, rating, did_rate)

tensor(0.3944, dtype=torch.float64, grad_fn=<DivBackward0>)

# **PREDICTIONS AND SAVING THE FILE**

In [0]:
predictions = torch.mm(movie_features, torch.t(user_prefs))

In [0]:
predictions = predictions + rating_mean

In [25]:
all_predictions = torch.tensor(predictions+0.5 , dtype=torch.long)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [26]:
print(all_predictions)

tensor([[5, 4, 4,  ..., 4, 3, 5],
        [3, 3, 3,  ..., 2, 2, 3],
        [4, 3, 3,  ..., 2, 2, 3],
        ...,
        [4, 4, 4,  ..., 4, 3, 4],
        [4, 3, 4,  ..., 4, 3, 4],
        [4, 4, 4,  ..., 4, 4, 4]])


In [0]:
all_predictions_arr = all_predictions.numpy()

In [0]:
result_frame = pd.DataFrame(all_predictions_arr, index=frame.index, columns = frame.columns)

In [29]:
result_frame

userId,156,208,359,572,586,741,768,775,903,982,...,136875,136989,137037,137202,137277,137343,137686,137885,138208,138325
movieId,,,,,,,,,,,,,,,,,,,,,
1,5,4,4,4,4,5,4,4,4,4,...,4,2,3,5,4,4,5,4,3,5
2,3,3,3,4,3,3,3,3,3,3,...,3,2,3,4,3,3,3,2,2,3
3,4,3,3,4,3,4,3,2,3,2,...,3,1,3,3,4,2,3,2,2,3
4,4,2,3,3,3,3,3,2,3,2,...,3,0,3,3,3,2,3,2,1,3
5,4,2,3,3,3,3,3,2,3,3,...,3,0,3,3,3,2,3,2,2,3
6,4,4,4,4,4,4,4,3,4,4,...,4,3,4,4,4,4,4,4,4,4
7,4,3,3,4,3,3,3,3,3,3,...,3,2,3,3,3,3,3,2,2,3
9,4,3,2,4,3,4,3,2,2,2,...,3,1,2,4,4,2,3,1,1,3
10,4,3,4,3,3,4,3,3,4,3,...,4,2,3,4,4,3,4,3,3,4


In [0]:
result_frame.to_csv('result_frame.csv')

# **ADD NEW RATINGS**

In [0]:
movie_features = movie_features.detach().numpy()
user_prefs = user_prefs.detach().numpy()

In [0]:
my_rating = np.zeros(shape=(1700,1))
my_did_rate = np.zeros(shape=(1700,1))

In [0]:
# lt=[]
# pp = frame.copy()
# pp['name']="abcd"
# pp.head()
# for i in frame.index:
#   if i in movie.movieId.values:
#     lt.append(movie[movie['movieId'] == i].iloc[0][1])
# pp['name']= lt
# pp.to_csv('rating_with_name.csv')

In [34]:
rating_with_name.head()

,movieId,156,208,359,572,586,741,768,775,903,...,136989,137037,137202,137277,137343,137686,137885,138208,138325,name
0,1,5.0,4.0,5.0,5.0,2.5,5.0,NaN,4.5,4.0,...,2.0,4.0,4.5,4.0,4.0,5.0,5.0,3.0,5.0,Toy Story (1995)
1,2,5.0,NaN,NaN,3.5,3.0,3.0,3.0,2.0,4.0,...,2.0,3.0,2.5,2.5,2.5,3.0,3.0,2.0,3.0,Jumanji (1995)
2,3,2.0,NaN,NaN,3.5,2.0,3.0,NaN,3.5,2.0,...,NaN,2.0,3.5,3.0,3.5,3.0,4.0,2.0,NaN,Grumpier Old Men (1995)
3,4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,Waiting to Exhale (1995)
4,5,3.0,NaN,NaN,3.5,3.0,4.0,2.0,1.0,3.0,...,4.5,3.0,NaN,3.0,3.5,NaN,3.0,2.0,NaN,Father of the Bride Part II (1995)


In [0]:
for i in range(100):
  my_rating[i]=5
  my_did_rate[i]=1

In [36]:
rating_with_name[rating_with_name['movieId']==81834]

,movieId,156,208,359,572,586,741,768,775,903,...,136989,137037,137202,137277,137343,137686,137885,138208,138325,name
1685,81834,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,NaN,Harry Potter and the Deathly Hallows: Part 1 (...


In [0]:
did_rate = np.append(did_rate, my_did_rate, axis=1)

In [0]:
train_data = np.append(train_data, my_rating, axis=1)

In [39]:
train_data.shape

(1700, 1001)

In [40]:
did_rate.shape

(1700, 1001)

In [0]:
my_prefs = np.random.randn(1,3)

In [0]:
user_prefs = np.append(user_prefs, my_prefs, axis=0)

In [43]:
user_prefs.shape

(1001, 3)

In [44]:
movie_features.shape

(1700, 3)

In [45]:
rating.shape

torch.Size([1700, 1000])

In [0]:
rating, rating_mean = normalize_ratings(train_data, did_rate)

In [48]:
rating.shape

(1700, 1001)

In [49]:
movie_features

array([[ 0.96973653, -0.50460597, -0.75990939],
       [ 0.15729144, -0.40599005, -0.52776559],
       [ 0.44038786, -1.06660895,  0.059473  ],
       ...,
       [ 0.39036299, -0.42745025,  0.47829839],
       [ 0.77360034, -0.40443659, -0.09544745],
       [ 0.74728106, -0.2381054 , -0.22614752]])

In [50]:
my_prefs

array([[ 1.62434536, -0.61175641, -0.52817175]])

In [0]:
movie_features = torch.tensor(movie_features, requires_grad=True)
user_prefs = torch.tensor(user_prefs, requires_grad=True)
did_rate = torch.tensor(did_rate, dtype=torch.double, requires_grad=False)
rating = torch.tensor(rating, requires_grad=False)
rating_mean = torch.tensor(rating_mean, dtype=torch.double, requires_grad=False)

In [52]:
loss(movie_features, user_prefs, rating, did_rate)

tensor(0.3945, dtype=torch.float64, grad_fn=<DivBackward0>)

In [53]:
did_rate

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 1., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)